<a href="https://colab.research.google.com/github/joe94113/Machine-Learning/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data from https://www.kaggle.com/c/titanic/data
import pandas as pd
train_df = pd.read_csv('train.csv', encoding="utf-8")
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
test_df = pd.read_csv("test.csv",encoding="utf-8")
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
train_df.isna().sum() # isna()，查看缺失值，有則False否則True

In [ ]:
test_df.isna().sum() 

In [ ]:
med = train_df.median() # 求出中位數
train_df = train_df.fillna(med) # 填補缺失值
test_df = test_df.fillna(med) # 測試資料要用訓練資料填補，中位數需要大量資料

In [ ]:
most_Embarked = train_df['Embarked'].value_counts().idxmax() # value_counts()查看value出現次數，idxmax()取出最多次數的
train_df['Embarked'] = train_df['Embarked'].fillna(most_Embarked)
test_df['Embarked'] = test_df['Embarked'].fillna(most_Embarked)

In [ ]:
def strcut(s):
  reserved = ["Mr", "Mrs", "Miss", "Master"]
  s = s.split(',')[-1].split('.')[0]
  s = s.strip()
  if s in reserved:
    return s
  else:
    return None
train_df["Name"] = train_df["Name"].apply(strcut)
test_df["Name"] = test_df["Name"].apply(strcut)

In [ ]:
# one-hot encoding
train = pd.get_dummies(train_df, columns=['Name', 'Sex', 'Embarked']) 
test = pd.get_dummies(test_df, columns=['Name', 'Sex', 'Embarked']) 

In [ ]:
train = train.drop(["PassengerId", 'Ticket', 'Cabin'], axis=1) # axis控制刪除直行或橫行
test = test.drop(['Ticket', 'Cabin'], axis=1)

In [ ]:
x_train = train.drop(["Survived"], axis=1)
y_train = train["Survived"]
x_predict = test.drop(["PassengerId"], axis=1)
pid = test['PassengerId']

In [ ]:
from sklearn.ensemble import RandomForestClassifier # 演算法:隨機森林
# 極盡所能的對所有指定的參數做搜尋，故稱之為暴力搜尋
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier()
params = {
    "n_estimators": range(20, 30),
    "max_depth": range(5, 8)
} # 持續跑縮小參數範圍
grid_search = GridSearchCV(
    clf,
    params,
    cv=10,
    n_jobs=4
)
# fit()求得訓練集X的均值啊，方差啊，最大值啊，最小值啊這些訓練集X固有的屬性。可以理解為一個訓練過程
grid_search.fit(x_train, y_train)
print("最佳參數:", grid_search.best_params_)
print("最佳分數:", grid_search.best_score_)

最佳參數: {'max_depth': 6, 'n_estimators': 27}
最佳分數: 0.838414481897628


In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score # 交叉驗證
# n_estimators幾顆森林構成
clf = RandomForestClassifier(n_estimators=20, max_depth=7)
# cv:分成幾份，n_jobs:平行執行，同時做幾份工作
scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=4)
print("十次分數", scores)
print("平均分數", np.average(scores))

十次分數 [0.77777778 0.80898876 0.74157303 0.91011236 0.88764045 0.80898876
 0.82022472 0.75280899 0.87640449 0.85393258]
平均分數 0.8238451935081148


In [ ]:
clf = RandomForestClassifier(n_estimators=20, max_depth=7)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
pre = clf.predict(x_predict)
result = pd.DataFrame({
    "PassengerId": pid,
    "Survived": pre
})
result.to_csv("titanic_rf.csv", encoding="utf-8", index=False)
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
